In [ ]:
import subprocess
import socket
import threading
import time
import os

# === 配置参数 ===
PORT = 8011
HOST = "192.168.50.253"


# === 导入并启动 http_server2.py 中的 run_server_select() ===
import http_server2  # 你需要确保 http_server2.py 在同一目录或 PYTHONPATH 中

def start_server(PORT):
    http_server2.run_server_select(PORT)

# # === 启动 server in background thread ===
# threading.Thread(target=start_server, daemon=True).start()
# print("[*] HTTP server started on port 8011")
# time.sleep(1)  # 等待服务器启动

# === 调用 server 函数（可调试） ===
print(f"[i] Calling run_server_debug({PORT}) from Jupyter...")
start_server(PORT)

# === 创建测试文件 ===
with open("index.html", "w") as f:
    f.write("<html><body><h1>Hello from http_server2</h1></body></html>")

with open("test.txt", "w") as f:
    f.write("Should not be served.")

# === curl 测试封装 ===
def curl_test(label, url, method="GET"):
    print(f"\n=== {label} ===")
    cmd = ["curl", "-i", "-X", method, url]
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout.strip())

# === telnet 挂起连接模拟 ===
def hold_connection():
    s = socket.socket()
    s.connect(("localhost", 8011))
    print("[*] Telnet-like connection held")
    time.sleep(5)
    s.close()
    print("[*] Telnet-like connection closed")

# === 逐项测试 ===
curl_test("✅ Valid GET (200)", "http://localhost:8011/index.html")
curl_test("❌ GET non-existent file (404)", "http://localhost:8011/does_not_exist.html")
curl_test("❌ GET unsupported file (403)", "http://localhost:8011/test.txt")
curl_test("❌ POST method (403)", "http://localhost:8011/index.html", method="POST")

# 并发连接测试
print("\n=== 🧪 Concurrent telnet + curl ===")
threading.Thread(target=hold_connection, daemon=True).start()
time.sleep(1)
curl_test("✅ curl during held telnet (should succeed)", "http://localhost:8011/index.html")

# 空请求连接测试
print("\n=== ❌ Empty Request ===")
s = socket.socket()
s.connect(("localhost", 8011))
s.close()
print("[*] Empty connection opened and closed")


[i] Calling run_server_debug(8011) from Jupyter...
[*] Listening on port 8011...
[+] Accepted connection from ('192.168.50.10', 57806)
[+] Accepted connection from ('192.168.50.10', 53155)
[DEBUG] Raw Request:
 GET /rfc2616.html HTTP/1.1
Host: 192.168.50.253:8011
Connection: keep-alive
DNT: 1
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6
Cookie: username-192-168-50-253-8888="2|1:0|10:1747406657|28:username-192-168-50-253-8888|200:eyJ1c2VybmFtZSI6ICIwZTlkMWQ4MzhjNDU0YzAyYTc3MmRhN2MxYjAwZjc0OCIsICJuYW1lIjogIkFub255bW91cyBDYWxsaXN0byIsICJkaXNwbGF5X25hbWUiOiAiQW5vbnltb3VzIENhbGxpc3RvIiwgImluaXRpYWxzIjogIkFDIiwgImNvbG9yIjogbnVsbH0=|b11d32b18f16